### OpenMP and MPI Interactive Tutorial

## Part 2: MPI

### 7. Introduction to MPI

#### What is MPI?
MPI (Message Passing Interface) is a standardized and portable message-passing system designed for high-performance computing (HPC) applications running on distributed-memory systems. It enables processes to communicate with each other by sending and receiving messages.

#### Key Features of MPI:
- Designed for distributed-memory parallel computing.
- Provides communication between processes running on different nodes.
- Offers both point-to-point and collective communication.
- Highly scalable for large-scale parallel applications.
- Portable across different computing architectures.

#### MPI vs OpenMP
| Feature        | MPI (Message Passing Interface) | OpenMP (Open Multi-Processing) |
|---------------|--------------------------------|--------------------------------|
| Memory Model  | Distributed                     | Shared                         |
| Communication | Explicit message passing        | Implicit (shared memory)       |
| Scalability   | Scales across multiple nodes    | Limited to a single node       |
| Synchronization | Explicit                       | Implicit                        |

#### MPI Implementations
Several implementations of MPI are widely used, including:
- **MPICH** (Argonne National Laboratory)
- **OpenMPI** (Open source, supports multiple platforms)
- **Intel MPI** (Optimized for Intel architectures)
- **MVAPICH** (Optimized for high-performance networks)

#### Installing MPI
To use MPI, you need an MPI implementation installed on your system.

**For Ubuntu/Linux:**
```bash
sudo apt update
sudo apt install mpich
```

**For macOS (using Homebrew):**
```bash
brew install mpich
```

**For Windows:**
- Use Windows Subsystem for Linux (WSL) and install MPI as shown above.
- Alternatively, install Microsoft MPI (MS-MPI).

#### Running an MPI Program
MPI programs typically run across multiple processes using `mpirun` or `mpiexec`.

Example:
```bash
mpicc my_mpi_program.c -o my_mpi_program
mpirun -np 4 ./my_mpi_program
```

In the next section, we will explore **Writing Your First MPI Program**!

### 8. Writing Your First MPI Program

MPI programs follow a standard structure consisting of initialization, communication, and finalization. Below, we will write a basic MPI program that demonstrates these concepts.

#### Basic MPI Program (Hello World)
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    printf("Hello from process %d out of %d\n", rank, size);
    
    MPI_Finalize();
    return 0;
}
```

#### Explanation:
1. **MPI_Init**: Initializes the MPI environment.
2. **MPI_Comm_rank**: Retrieves the rank (ID) of the current process.
3. **MPI_Comm_size**: Determines the total number of processes.
4. **MPI_Finalize**: Cleans up the MPI environment before program termination.

#### Compiling and Running the MPI Program
To compile the program, use an MPI compiler such as `mpicc`:
```bash
mpicc hello_mpi.c -o hello_mpi
```
Run the program with multiple processes:
```bash
mpirun -np 4 ./hello_mpi
```
Expected Output:
```
Hello from process 0 out of 4
Hello from process 1 out of 4
Hello from process 2 out of 4
Hello from process 3 out of 4
```

In [1]:
%%file hello_mpi.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    printf("Hello from process %d out of %d\n", rank, size);
    
    MPI_Finalize();
    return 0;
}

Overwriting hello_mpi.c


In [2]:
!mpicc hello_mpi.c -o hello_mpi
!mpiexec --allow-run-as-root -np 2 ./hello_mpi

Hello from process 0 out of 2
Hello from process 1 out of 2


#### Exercise 1:
Modify the Hello World program so that only the master process (rank 0) prints a message like:
```
Master process says: Hello, I manage 3 other processes!
```

This concludes the introduction to MPI programming. Next, we will explore **Point-to-Point Communication in MPI**!


In [3]:
%%file exercise_1.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    printf("TODO message only on master!");
    
    MPI_Finalize();
    return 0;
}

Overwriting exercise_1.c


In [4]:
!mpicc exercise_1.c -o exercise_1
!mpiexec --allow-run-as-root -np 2 ./exercise_1

TODO message only on master!TODO message only on master!

### 9. Point-to-Point Communication

Point-to-point communication in MPI allows processes to send and receive messages directly. It is the fundamental method for communication in distributed-memory systems.

#### 9.1 MPI Send and Receive
The basic functions for point-to-point communication are:
- `MPI_Send`: Sends a message from one process to another.
- `MPI_Recv`: Receives a message sent from another process.

#### Example: Basic Send and Receive
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    if (rank == 0) {
        int data = 100;
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Process 0 sent data %d to process 1\n", data);
    } else if (rank == 1) {
        int received_data;
        MPI_Recv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received data %d from process 0\n", received_data);
    }
    
    MPI_Finalize();
    return 0;
}
```

#### Explanation:
1. **Process 0** sends an integer (`data = 100`) to **Process 1** using `MPI_Send`.
2. **Process 1** receives the data using `MPI_Recv`.
3. The tag `0` is used to identify the message.
4. `MPI_STATUS_IGNORE` is used if message status details are not required.

#### Compiling and Running the Program
```bash
mpicc mpi_send_recv.c -o mpi_send_recv
mpirun -np 2 ./mpi_send_recv
```

Expected Output:
```
Process 0 sent data 100 to process 1
Process 1 received data 100 from process 0
```



In [33]:
%%file mpi_send_recv.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Get rank of process
    MPI_Comm_size(MPI_COMM_WORLD, &size); // Get total number of processes

    if (size < 2) {
        if (rank == 0) {
            printf("This program requires at least 2 processes!\n");
        }
        MPI_Finalize();
        return 0;
    }

    if (rank == 0) {
        int data = 42;
        printf("Process %d sending data %d to process 1\n", rank, data);
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
    } 
    else if (rank == 1) {
        int received_data;
        MPI_Recv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process %d received data %d from process 0\n", rank, received_data);
    }

    MPI_Finalize();
    return 0;
}


Writing mpi_send_recv.c


In [34]:
!mpicc mpi_send_recv.c -o mpi_send_recv
!mpiexec --allow-run-as-root -np 2 ./mpi_send_recv

Process 1 received data 42 from process 0
Process 0 sending data 42 to process 1


#### 9.2 Blocking vs. Non-Blocking Communication
MPI provides two types of send/receive operations:
- **Blocking:** The sender waits until the message is received (`MPI_Send`, `MPI_Recv`).
- **Non-Blocking:** The sender continues execution without waiting (`MPI_Isend`, `MPI_Irecv`).

Example of Non-Blocking Communication:
```c
MPI_Request request;
MPI_Isend(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD, &request);
```

#### Exercise 2:
Modify the program to:
- Allow **Process 1** to send a message back to **Process 0**.
- Use **non-blocking communication (`MPI_Isend` and `MPI_Irecv`)** instead of blocking calls.

This concludes the introduction to **Point-to-Point Communication in MPI**. Next, we will explore **Collective Communication in MPI**!


In [7]:
%%file exercise_2.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    printf("TODO implement non-blocking");

    MPI_Finalize();
    return 0;
}

Overwriting exercise_2.c


In [8]:
!mpicc exercise_2.c -o exercise_2
!mpiexec --allow-run-as-root -np 2 ./exercise_2

TODO implement non-blockingTODO implement non-blocking

### 10. Collective Communication

Collective communication in MPI allows multiple processes to communicate in a coordinated manner. These operations are optimized for efficiency and are useful for distributing data, gathering results, and synchronizing processes.

#### 10.1 Common Collective Communication Operations
MPI provides several collective communication functions:
- **MPI_Bcast**: Broadcasts data from one process to all other processes.
- **MPI_Scatter**: Distributes chunks of data from one process to all others.
- **MPI_Gather**: Collects data from all processes to a single process.
- **MPI_Reduce**: Combines values from all processes using an operation (e.g., sum, max, min).
- **MPI_Allreduce**: Like `MPI_Reduce`, but results are distributed to all processes.
- **MPI_Barrier**: Synchronizes all processes, ensuring they all reach the same execution point before proceeding.

---
### 10.2 Example: Broadcasting Data Using `MPI_Bcast`
The `MPI_Bcast` function allows one process to send data to all others in the communicator.
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank;
    int data = 0;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    if (rank == 0) {
        data = 100;
        printf("Process %d broadcasting data: %d\n", rank, data);
    }
    
    MPI_Bcast(&data, 1, MPI_INT, 0, MPI_COMM_WORLD);
    printf("Process %d received data: %d\n", rank, data);
    
    MPI_Finalize();
    return 0;
}
```


In [9]:
%%file broadcasting.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank;
    int data = 0;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    if (rank == 0) {
        data = 100;
        printf("Process %d broadcasting data: %d\n", rank, data);
    }
    
    MPI_Bcast(&data, 1, MPI_INT, 0, MPI_COMM_WORLD);
    printf("Process %d received data: %d\n", rank, data);
    
    MPI_Finalize();
    return 0;
}

Overwriting broadcasting.c


In [10]:
!mpicc broadcasting.c -o broadcasting
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./broadcasting

Process 3 received data: 100
Process 0 broadcasting data: 100
Process 0 received data: 100
Process 1 received data: 100
Process 2 received data: 100


### 10.3 Example: Gathering Data Using `MPI_Gather`
Each process sends a value to a root process, which collects all values.
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int local_value = rank * 10;
    int gathered_values[size];
    
    MPI_Gather(&local_value, 1, MPI_INT, gathered_values, 1, MPI_INT, 0, MPI_COMM_WORLD);
    
    if (rank == 0) {
        printf("Gathered values: ");
        for (int i = 0; i < size; i++) {
            printf("%d ", gathered_values[i]);
        }
        printf("\n");
    }
    
    MPI_Finalize();
    return 0;
}
```

In [11]:
%%file mpi_gather.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int local_value = rank * 10;
    int gathered_values[size];
    
    MPI_Gather(&local_value, 1, MPI_INT, gathered_values, 1, MPI_INT, 0, MPI_COMM_WORLD);
    
    if (rank == 0) {
        printf("Gathered values: ");
        for (int i = 0; i < size; i++) {
            printf("%d ", gathered_values[i]);
        }
        printf("\n");
    }
    
    MPI_Finalize();
    return 0;
}

Overwriting mpi_gather.c


In [12]:
!mpicc mpi_gather.c -o mpi_gather
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./mpi_gather

Gathered values: 0 10 20 30 


### 10.4 Example: Reducing Values Using `MPI_Reduce`
`MPI_Reduce` applies an operation (sum, max, min, etc.) across all processes and returns the result to the root.
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int local_value = rank + 1;
    int sum;
    
    MPI_Reduce(&local_value, &sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    
    if (rank == 0) {
        printf("Total sum: %d\n", sum);
    }
    
    MPI_Finalize();
    return 0;
}
```

In [13]:
%%file mpi_reduce.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int local_value = rank + 1;
    int sum;
    
    MPI_Reduce(&local_value, &sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    
    if (rank == 0) {
        printf("Total sum: %d\n", sum);
    }
    
    MPI_Finalize();
    return 0;
}

Overwriting mpi_reduce.c


In [14]:
!mpicc mpi_reduce.c -o mpi_reduce
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./mpi_reduce

Total sum: 10


### 🎯 Exercise 3
Modify the `MPI_Reduce` example to find the **maximum** value among all ranks using `MPI_MAX`.

This concludes the **Collective Communication** section. Next, we will explore **Advanced MPI Concepts**!

In [15]:
%%file exercise_3.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    printf("TODO implement me!\n");
    
    MPI_Finalize();
    return 0;
}

Overwriting exercise_3.c


In [16]:
!mpicc exercise_3.c -o exercise_3
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./exercise_3

TODO implement me!
TODO implement me!
TODO implement me!
TODO implement me!


### 11. MPI Derived Data Types and Communicators

MPI allows users to define **custom data types** for efficient message passing and to create **new communicators** for better process organization.

---
### 11.1 MPI Derived Data Types
By default, MPI supports basic data types (`MPI_INT`, `MPI_FLOAT`, `MPI_DOUBLE`, etc.), but for complex data structures, **derived data types** can be created.

#### Creating a Struct Data Type
Example: Sending a struct with an `id` and `value` field.
```c
#include <mpi.h>
#include <stdio.h>

typedef struct {
    int id;
    double value;
} Data;

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    Data my_data;
    if (rank == 0) {
        my_data.id = 42;
        my_data.value = 3.14;
        MPI_Send(&my_data, 1, MPI_BYTE, 1, 0, MPI_COMM_WORLD);
        printf("Process %d sent data: id=%d, value=%.2f\n", rank, my_data.id, my_data.value);
    } else if (rank == 1) {
        MPI_Recv(&my_data, 1, MPI_BYTE, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process %d received data: id=%d, value=%.2f\n", rank, my_data.id, my_data.value);
    }
    
    MPI_Finalize();
    return 0;
}
```

#### Using `MPI_Type_create_struct`
For better portability, it is recommended to use the `MPI_Datatype` struct. For example:
```c
MPI_Datatype MPI_DATA_TYPE;
int block_lengths[2] = {1, 1};
MPI_Aint offsets[2];
offsets[0] = offsetof(Data, id);
offsets[1] = offsetof(Data, value);
MPI_Datatype types[2] = {MPI_INT, MPI_DOUBLE};
MPI_Type_create_struct(2, block_lengths, offsets, types, &MPI_DATA_TYPE);
MPI_Type_commit(&MPI_DATA_TYPE);
```


In [17]:
%%file struct.c
#include <mpi.h>
#include <stdio.h>

typedef struct {
    int id;
    double value;
} Data;

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    Data my_data;
    if (rank == 0) {
        my_data.id = 42;
        my_data.value = 3.14;
        MPI_Send(&my_data, 1, MPI_BYTE, 1, 0, MPI_COMM_WORLD);
        printf("Process %d sent data: id=%d, value=%.2f\n", rank, my_data.id, my_data.value);
    } else if (rank == 1) {
        MPI_Recv(&my_data, 1, MPI_BYTE, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process %d received data: id=%d, value=%.2f\n", rank, my_data.id, my_data.value);
    }
    
    MPI_Finalize();
    return 0;
}

Overwriting struct.c


In [18]:
!mpicc struct.c -o struct
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./struct

Process 0 sent data: id=42, value=3.14
Process 1 received data: id=42, value=0.00


### 11.2 MPI Communicators
Communicators define **groups of processes** that can communicate. By default, all processes belong to `MPI_COMM_WORLD`, but custom communicators allow fine-grained control.

#### Creating a New Communicator
Example: Splitting MPI_COMM_WORLD into two groups.
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, color;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    color = rank % 2; // Create two groups (even, odd ranks)
    MPI_Comm new_comm;
    MPI_Comm_split(MPI_COMM_WORLD, color, rank, &new_comm);
    
    int new_rank;
    MPI_Comm_rank(new_comm, &new_rank);
    printf("Old Rank: %d, New Rank: %d, Group: %d\n", rank, new_rank, color);
    
    MPI_Finalize();
    return 0;
}
```
#### Explanation:
- `MPI_Comm_split()` creates **two new communicators** based on `color` (even/odd ranks).
- Each new group has its own rank assignment (`new_rank`).
- This allows better communication **within subgroups**.



In [19]:
%%file communicator.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, color;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    color = rank % 2; // Create two groups (even, odd ranks)
    MPI_Comm new_comm;
    MPI_Comm_split(MPI_COMM_WORLD, color, rank, &new_comm);
    
    int new_rank;
    MPI_Comm_rank(new_comm, &new_rank);
    printf("Old Rank: %d, New Rank: %d, Group: %d\n", rank, new_rank, color);
    
    MPI_Finalize();
    return 0;
}

Overwriting communicator.c


In [20]:
!mpicc communicator.c -o communicator
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./communicator

Old Rank: 1, New Rank: 0, Group: 1
Old Rank: 2, New Rank: 1, Group: 0
Old Rank: 3, New Rank: 1, Group: 1
Old Rank: 0, New Rank: 0, Group: 0


### 🎯 **Exercise**
Modify the **derived data type example** to create an `MPI_Type_create_struct()` and send/receive data properly using the new type.

This concludes **MPI Derived Data Types and Communicators**. Next, we will explore **Advanced MPI Concepts**!


In [21]:
%%file exercise_4.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    printf("TODO implement me!\n");
    
    MPI_Finalize();
    return 0;
}

Overwriting exercise_4.c


In [22]:
!mpicc exercise_4.c -o exercise_4
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./exercise_4

TODO implement me!
TODO implement me!
TODO implement me!
TODO implement me!


### 12. Hands-on Exercises for MPI

To reinforce MPI concepts, here are several hands-on exercises covering communication, collective operations, and derived data types.

---
### 12.1 Exercise 5: Basic Point-to-Point Communication
**Task:** Implement an MPI program where:
- Process 0 sends an integer to Process 1.
- Process 1 receives it and prints the value.

**Code:**
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    if (rank == 0) {
        int data = 42;
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Process 0 sent data %d to Process 1\n", data);
    } else if (rank == 1) {
        int received_data;
        MPI_Recv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received data %d from Process 0\n", received_data);
    }
    
    MPI_Finalize();
    return 0;
}
```

In [23]:
%%file exercise_5.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    
    if (rank == 0) {
        int data = 42;
        MPI_Send(&data, 1, MPI_INT, 1, 0, MPI_COMM_WORLD);
        printf("Process 0 sent data %d to Process 1\n", data);
    } else if (rank == 1) {
        int received_data;
        MPI_Recv(&received_data, 1, MPI_INT, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
        printf("Process 1 received data %d from Process 0\n", received_data);
    }
    
    MPI_Finalize();
    return 0;
}

Writing exercise_5.c


In [24]:
!mpicc exercise_5.c -o exercise_5
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./exercise_5

Process 1 received data 42 from Process 0
Process 0 sent data 42 to Process 1


### 12.2 Exercise 6: Collective Communication using `MPI_Bcast`
**Task:** Modify the program so that:
- Process 0 initializes an integer variable.
- All processes receive the same value using `MPI_Bcast`.

**Hint:** Use `MPI_Bcast(&data, 1, MPI_INT, 0, MPI_COMM_WORLD);`.

In [25]:
%%file exercise_6.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    printf("TODO implement me!\n");
    
    MPI_Finalize();
    return 0;
}

Overwriting exercise_6.c


In [26]:
!mpicc exercise_6.c -o exercise_6
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./exercise_6

TODO implement me!
TODO implement me!
TODO implement me!
TODO implement me!


### 12.3 Exercise 7: Summation using `MPI_Reduce`
**Task:** Implement an MPI program where each process:
- Computes a local sum based on its rank.
- Uses `MPI_Reduce` to compute the total sum at Process 0.

**Code:**
```c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int local_value = rank + 1;
    int total_sum;
    
    MPI_Reduce(&local_value, &total_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    
    if (rank == 0) {
        printf("Total sum: %d\n", total_sum);
    }
    
    MPI_Finalize();
    return 0;
}
```

In [27]:
%%file exercise_7.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);
    
    int rank, size;
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);
    
    int local_value = rank + 1;
    int total_sum;
    
    MPI_Reduce(&local_value, &total_sum, 1, MPI_INT, MPI_SUM, 0, MPI_COMM_WORLD);
    
    if (rank == 0) {
        printf("Total sum: %d\n", total_sum);
    }
    
    MPI_Finalize();
    return 0;
}

Overwriting exercise_7.c


In [28]:
!mpicc exercise_7.c -o exercise_7
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./exercise_7

Total sum: 10


### 12.4 Exercise 8: Custom Data Type Communication
**Task:**
- Define a custom struct with an `id` and a `value`.
- Use `MPI_Type_create_struct` to send and receive the struct between processes.

**Hint:**
- Use `MPI_Type_create_struct()` to define a derived type.
- Use `MPI_Send` and `MPI_Recv` to send/receive struct data.



In [29]:
%%file exercise_8.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    printf("TODO implement me!\n");
    
    MPI_Finalize();
    return 0;
}

Writing exercise_8.c


In [30]:
!mpicc exercise_8.c -o exercise_8
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./exercise_8

TODO implement me!
TODO implement me!
TODO implement me!
TODO implement me!


### 🎯 Bonus Exercise: Ring Communication
Modify the program so that:
- Each process sends data to the **next process** in a ring.
- The last process sends back to process 0.

This concludes the **MPI Hands-on Exercises** section. Happy coding! 🚀

In [31]:
%%file exercise_9.c
#include <mpi.h>
#include <stdio.h>

int main(int argc, char** argv) {
    MPI_Init(&argc, &argv);

    printf("TODO implement me!\n");
    
    MPI_Finalize();
    return 0;
}

Writing exercise_9.c


In [32]:
!mpicc exercise_9.c -o exercise_9
!mpiexec --allow-run-as-root --oversubscribe  -np 4 ./exercise_9

TODO implement me!
TODO implement me!
TODO implement me!
TODO implement me!
